运动模糊分析：根据相机帧间（线速度v，角速度w），以及帧间深度图差(配准后的) diffDmap，以及法向图 nmap
--
@2017-11-30 17:11:32


In [1]:
                                    %pylab
import sys
sys.path.append(r'D:\opencv249\sources\modules\python\test')

import os
import cv2
import transformations as tf
'一些短别名:'
tf.e2m = tf.euler_matrix
tf.m2e = tf.euler_from_matrix
tf.q2m = tf.quaternion_matrix
tf.m2q = tf.quaternion_from_matrix
tf.e2q = tf.quaternion_from_euler
tf.q2e = tf.euler_from_quaternion
tf.m2r = tf.rotation_from_matrix
tf.r2m = tf.rotation_matrix
tf.qinv = tf.quaternion_inverse
tf.qmul = tf.quaternion_multiply

'angle <--> radian:'
a2r = np.pi / 180
r2a = 180 / np.pi

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [23]:
fn1=r'D:\Users\zhangxaochen\Desktop\dbg\f2mkr-amp4\kinfu_poses.csv'
m1 = loadtxt(fn1, delimiter=',')[:, :7]
m1tvecs = m1[:, :3]
m1rvecs = m1[:, 3:]
m1rmats = np.array([tf.q2m([q[1], q[2], q[3], q[0]]) for q in m1rvecs]) #N*(4x4)

m1t0 = m1tvecs[0]
m1r0 = m1rmats[0]

tvecIntv=norm(m1tvecs[1:]-m1tvecs[:-1], axis=1) #帧间 interval
plot(tvecIntv)
'↑-发现: 相机运动(线速度)大的时刻, 物体在相机坐标系运动未必大, 还得看角速度'

# m1rmats = np.array([m1r0.dot(ri.T) for ri in m1rmats]) #R0*Ri.T
'此处按 pose是 c2g 来处理, 之前 compareEulerAngle.ipynb 里, 注释是 g2c, 记不清了(可能弄 cboard 时的定义?), 震惊! @2017-12-2 21:09:03'

#Ri'*R(i+1)
dRmats = [m1rmats[i].T.dot(m1rmats[i+1]) for i in range(len(m1rmats)-1)]
angles = []
for drm in dRmats:
    drvec, _ = cv2.Rodrigues(drm[:3, :3]) #rot-vec
    angles.append(norm(drvec))
    pass

plot(angles)
legend(['dt', 'dR'])

In [222]:
'tmp: 看 tcurr, tprev 在我的新权重函数下的影响'
tc_min, tc_max = -1, 10
tp_min, tp_max = -1, 10
tcurr=linspace(tc_min, tc_max, 201)
tprev=linspace(tp_min, tp_max, 201)
xv, yv = meshgrid(tcurr, tprev, sparse=True) #tcurr 横轴
'NO isNewFace 情形'
# myfunc = vectorize(lambda p, c: min(1, p / min(1, abs(c))) ) #错 min 会向下转型, float->int 导致错
myfunc = lambda p, c: max(1., p / max(1, abs(c))) #max(1, tsdf_prev1 / max(1, abs(tsdf_curr)) ); 
myfunc = lambda p, c: min(p, max(0., p / c) ) #min(tsdf_prev1, max(0, tsdf_prev1 / tsdf_curr) )
# myfunc = lambda p, c: max(0., p / c) #对比说明上面 min(p, ...) 必要性
# myfunc = lambda p, c: p / max(1, abs(c)) #等价:== min(p, p/c) 
# myfunc = lambda p, c: min(p, p/abs(c))

'YES isNewFace 情形'
# myfunc = lambda p, c: min(max(1., p), max(0., p/c) ) #p>0 时：min(max(1.f, tsdf_prev1), max(0.f, tsdf_prev1 / tsdf_curr) );
myfunc = lambda p, c: min(max(1., p), max(0., p/(c+np.sign(c)*0.01)) ) #p>0 时... 同上, 只是避免除零
myfunc = lambda p, c: -p/c if c+p>0 else (c+1)/max(p+1, 0.1) #-1<p<0 时：tsdf_curr + tsdf_prev1 > 0 ? 1 : (tsdf_curr+1) / max(tsdf_prev1+1, 0.1);
myfunc = lambda p, c: min(max(1., p), max(0., p/(c+np.sign(c)*0.01)) ) if p>0\
                else (-p/c if c+p>0 else (c+1)/max(p+1, 0.1)) #上面俩合并写

'tmp, test 测试 myfunc 一些形式:'
# myfunc = lambda p, c:
# myfunc = lambda p, c: -p/c


myfunc_vrize = vectorize(myfunc)
z = myfunc_vrize(yv, xv)
# z.shape #(200L, 200L), 是 y*x
# z[z==1] = 10 #故意把 w=1 区域弄成明显颜色
# z[abs(z-1)<1e-4] = 10 

figure()
# xlim(tc_min, tc_max) #错, 用 imshow 的 extent
# ylim(tp_min, tp_max)
# imshow(z, interpolation=None, extent=(tc_min, tc_max, tp_max, tp_min)) #left, right, bot, top
imshow(z[::-1], interpolation=None, extent=(tc_min, tc_max, tp_min, tp_max)) #z 纵轴倒置
colorbar() #放在 imshow 之后


<matplotlib.colorbar.Colorbar instance at 0x000000004E059388>

In [116]:
z[100, 30]

5.025125628140704

In [211]:
print myfunc(2, 1.5)
print myfunc(0.9, 0.3)
print myfunc(0.3, 0.9)
print myfunc(0.5, 0.5)
print myfunc(1.2, 0.5)
print myfunc(6, 3)
print myfunc(6, 4)
print myfunc(-0.8, 1)
print myfunc(-0.9, 0.8)


1.32450331126
1.0
0.32967032967
0.980392156863
1.2
1.99335548173
1.49625935162
0.8
18.0


In [35]:
'demo: meshgrid'
x = np.arange(-5, 5, 0.1)
y = np.arange(-5, 5, 0.1)
xx, yy = np.meshgrid(x, y, sparse=True)
z = np.sin(xx**2 + yy**2) / (xx**2 + yy**2)
h = plt.contourf(x,y,z)
# show()
colorbar()

<matplotlib.colorbar.Colorbar instance at 0x0000000018C5A1C8>

In [226]:
'demo: 高斯, 正态曲线'
import pandas as pd # 导入另一个包“pandas” 命名为 pd，理解成pandas是在 numpy 基础上的升级包
import numpy as np #导入一个数据分析用的包“numpy” 命名为 np
import matplotlib.pyplot as plt # 导入 matplotlib 命名为 plt，类似 matlab，集成了许多可视化命令

#jupyter 的魔术关键字（magic keywords）
#在文档中显示 matplotlib 包生成的图形
# 设置图形的风格
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

data = pd.read_csv('stakes.csv') #载入数据文件
data.head(5) #查看前5个头部数据

IOError: File stakes.csv does not exist

In [33]:
colorbar?